## Task 2: CORE SECTION (ICP Algorithm)

In [1]:
import sys,os

# Path to Resources and Python Libraries
RES_PATH = './bunny/data'
LIB_PATH = './python_lib'

# Checking if the path is found locally
if not os.path.exists(RES_PATH):
    print( 'cannot find mesh resources dir, please update RES_PATH')
    exit(1)
else:
    print('found resources')

# Loading Geo Tools
sys.path.append(LIB_PATH) 
from geo_tools import rd_helper

# Loading libraries for visualisation
import pyglet
pyglet.options['shadow_window'] = False
import pyrender
import matplotlib
import matplotlib.pyplot as plt

# Loading libraries for basic geometry processing
import trimesh
import numpy as np
from sklearn.neighbors import KDTree

found resources


### Implement ICP Algorithm 

In [5]:
def icp(M1, M2, max_iterations=100, tolerance=1e-6):
    """
    Perform Iterative Closest Point (ICP) algorithm to align M2 to M1.

    Args:
    - M1: numpy array representing the vertices of model M1 (shape: Nx3)
    - M2: numpy array representing the vertices of model M2 (shape: Nx3)
    - max_iterations: maximum number of iterations for ICP algorithm
    - tolerance: convergence threshold

    Returns:
    - R: 3x3 rotation matrix
    - t: 3x1 translation vector
    - registered_models: list of models after each iteration (as trimesh.pointcloud.PointCloud objects)
    """

    M1_vertices = M1.vertices.copy()
    M2_vertices = M2.vertices.copy()

    # Initialize rotation matrix R and translation vector t
    R = np.eye(3)
    t = np.zeros((3, 1))

    # Initialize list to store registered models
    registered_models = []

    for iteration in range(1, max_iterations+1):
        # Find the nearest neighbors between M1 and transformed M2
        tree = KDTree(M1_vertices)
        _, indices = tree.query(M2_vertices)

        # Extract corresponding points from M1 and M2
        P = M1_vertices[indices].squeeze()
        Q = M2_vertices

        # Compute the centroid of the matched points
        centroid_P = np.mean(P, axis=0, keepdims=True)  #(1, 3)
        centroid_Q = np.mean(Q, axis=0, keepdims=True)  #(1, 3)

        # Compute the cross-covariance matrix
        A = (Q - centroid_Q).T @ (P - centroid_P)   #(3, n) x (n, 3)

        # Use Singular Value Decomposition (SVD) to compute the optimal rotation matrix R
        U, _, Vt = np.linalg.svd(A)
        R = Vt.T @ np.diag([1, 1, np.linalg.det(Vt.T @ U.T)]) @ U.T

        # Compute the optimal translation vector t
        t = centroid_P.T - R @ centroid_Q.T

        # Apply transformation to M2 for the next iteration and store the result
        M2_vertices = (R @ M2_vertices.T + t).T
        registered_model = trimesh.points.PointCloud(M2_vertices)
        registered_models.append(registered_model)

        # Check convergence
        if np.linalg.norm(t) < tolerance:
            print('Converge at iteration', iteration)
            break

    if iteration == max_iterations:
        print(iteration, 'the iteration reached maximum.')
    
    return R, t, registered_models


### Loading the point cloud model M1 and M2

In [3]:
# Loading the point cloud model M1 and M2
mesh_fp1 = os.path.join(RES_PATH,'bun000.ply')
assert os.path.exists(mesh_fp1), 'Cannot find:' + mesh_fp1
M1 = trimesh.load(mesh_fp1)
print('M1 shape of Vertices:', M1.vertices.shape)

mesh_fp2 = os.path.join(RES_PATH,'bun045.ply')
assert os.path.exists(mesh_fp2), 'Cannot find:' + mesh_fp2
M2 = trimesh.load(mesh_fp2)
print('M2 shape of Vertices:', M2.vertices.shape)

M1 shape of Vertices: (40256, 3)
M2 shape of Vertices: (40097, 3)


### Using ICP algorithm to do local registration from M2 to M1

In [7]:
# Conduct ICP algorithm...
R, t, registered_models = icp(M1, M2)

# Save registered models
for i, model in enumerate(registered_models):
        directory = './results_task2/standard_ICP_results/'
        if not os.path.exists(directory):
            os.makedirs(directory)
        filename = f"registered_model_{i+1}.ply"
        filepath = os.path.join(directory, filename)
        model.export(filepath)

Converge at iteration 60


In [9]:
# Find non-overlapping points (after alignment)
registered_M2 = registered_models[-1]
tree = KDTree(M1.vertices)
distances, _ = tree.query(registered_M2.vertices)
non_overlapping_indices = np.where(np.isnan(distances))[0]
non_overlapping_points = registered_M2.vertices[non_overlapping_indices]
# Save non-overlapping points to a PLY file
trimesh.points.PointCloud(non_overlapping_points).export('./results_task2/standard_ICP_results/non_overlapping_points.ply')

b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 0\nproperty float x\nproperty float y\nproperty float z\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nproperty uchar alpha\nend_header\n'

### Check performance with perturbation
a) Assume that the model M1 is at the origin (i.e., centroid of M1 is at the origin). Now assume M2 = R(M1), i.e., a rotated version of model M1 about the origin. Progressively perturb the initial rotation of R and evaluate the convergence behavior of ICP trying to align M2 → M1. R can be rotation about any of the coordinate axes for example. One way to simulate the effect of increasing misalignment (i.e., initial alignment) is to rotate the object about z-axis over increasing rotation angles (say ±5, ±10, ±15,.. degrees).

In [10]:
def generate_rotation_matrix(angle):
    """
    Generate a rotation matrix for rotating around the z-axis by the given angle.

    Args:
    - angle: rotation angle in degrees

    Returns:
    - R: 3x3 rotation matrix
    """
    angle_rad = np.radians(angle)
    cos_theta = np.cos(angle_rad)
    sin_theta = np.sin(angle_rad)
    R = np.array([[cos_theta, -sin_theta, 0],
                  [sin_theta, cos_theta, 0],
                  [0, 0, 1]])
    return R

In [12]:
# Initialse model M1 at origin
M1_origin = M1.copy()
centroid_M1 = np.mean(M1_origin.vertices, axis=0, keepdims=True)  #(1, 3)
M1_origin.vertices = M1_origin.vertices - centroid_M1
directory = f'./results_task2/perturb_results/perturb by rotation/'
if not os.path.exists(directory):
    os.makedirs(directory)
filename = f"M1_origin.ply"
filepath = os.path.join(directory, filename)
M1_origin.export(filepath)

# Increase rotation angle progressively
perturb_angles = [5, 10, 15]    
for angle in perturb_angles:
    M2_origin = M1_origin.copy()
    print('Perturb', angle, 'degree...')
    # Generate rotation matrix for current angle
    R = generate_rotation_matrix(angle)

    # Rotate model M1 to generate model M2
    M2_origin.vertices = (R @ M1_origin.vertices.T).T

    # Run ICP to align M2 to M1
    _, _, registered_models = icp(M1_origin, M2_origin)

    for i, model in enumerate(registered_models):
        directory = f'./results_task2/perturb_results/perturb by rotation/perturb {angle} degree/'
        if not os.path.exists(directory):
            os.makedirs(directory)
        filename = f"registered_model_{i+1}.ply"
        filepath = os.path.join(directory, filename)
        model.export(filepath)

Perturb 5 degree...
Converge at iteration 19
Perturb 10 degree...
Converge at iteration 25
Perturb 15 degree...
Converge at iteration 28


b) Perturb model M2 (without the extra rotation) to simulate a noisy model say M2'. Evaluate how well ICP
performs as you continue to add more noise. As for noise, add zero-mean Gaussian noise – you can simply
perturb each vertex of the mesh M2 under this Gaussian model. Adjust the amount of noise based on the
bounding box dimensions of M2.

In [13]:
# Initialse model M1 at origin
M1_origin = M1.copy()
centroid_M1 = np.mean(M1_origin.vertices, axis=0, keepdims=True)  #(1, 3)
M1_origin.vertices = M1_origin.vertices - centroid_M1

# Increase std of Gaussian noise distribution progressively
noise_stddev_scales = [0.01, 0.02, 0.04]
for noise_stddev_scale in noise_stddev_scales:
    M2_origin = M1_origin.copy()
    # Perturb model M2 with Gaussian noise
    noise_scale = noise_stddev_scale * np.min(M1_origin.bounding_box.extents)  # Adjust the amount of noise based on bounding box dimensions
    M2_origin.vertices += np.random.normal(0, noise_scale, M2_origin.vertices.shape)
    print('Perturb', noise_scale, 'std...')

    # Run ICP to align M2_perturbed to M1
    _, _, registered_models = icp(M1_origin, M2_origin)

    for i, model in enumerate(registered_models):
        directory = f'./results_task2/perturb_results/perturb by noise/Gaussian (std = {noise_scale})/'
        if not os.path.exists(directory):
            os.makedirs(directory)
        filename = f"registered_model_{i+1}.ply"
        filepath = os.path.join(directory, filename)
        model.export(filepath)

Perturb 0.001174210011959076 std...
Converge at iteration 11
Perturb 0.002348420023918152 std...
Converge at iteration 19
Perturb 0.004696840047836304 std...
Converge at iteration 26


### Check performance with subsampling

Instead of directly aligning M2 to M1, speed up your computation by estimating the aligning transform using subsampled versions of M1 and/or M2 as appropriate. Report accuracy with increasing subsampling rates.

In [16]:
def icp_with_subsampling(M1, M2, subsample_rate, max_iterations=100, tolerance=1e-6):
    """
    Perform Iterative Closest Point (ICP) algorithm with subsampling to align M2 to M1.

    Args:
    - M1: numpy array representing the vertices of model M1 (shape: Nx3)
    - M2: numpy array representing the vertices of model M2 (shape: Nx3)
    - subsample_rate: subsampling rate (between 0 and 1)
    - max_iterations: maximum number of iterations for ICP algorithm
    - tolerance: convergence threshold

    Returns:
    - R: 3x3 rotation matrix
    - t: 3x1 translation vector
    - registered_models: list of models after each iteration (as trimesh.pointcloud.PointCloud objects)
    """

    # Subsample M1 and M2
    M1_vertices = M1.vertices.copy()[::int(1/subsample_rate)]
    M2_vertices = M2.vertices.copy()[::int(1/subsample_rate)]

    # Initialize rotation matrix R and translation vector t
    R = np.eye(3)
    t = np.zeros((3, 1))

    # Initialize list to store registered models
    registered_models = []

    for iteration in range(1, max_iterations+1):
        # Find the nearest neighbors between subsampled M1 and transformed subsampled M2
        tree = KDTree(M1_vertices)
        _, indices = tree.query(M2_vertices)

        # Extract corresponding points from subsampled M1 and M2
        P = M1_vertices[indices].squeeze()
        Q = M2_vertices

        # Compute the centroid of the matched points
        centroid_P = np.mean(P, axis=0, keepdims=True)  #(1, 3)
        centroid_Q = np.mean(Q, axis=0, keepdims=True)  #(1, 3)

        # Compute the cross-covariance matrix
        A = (Q - centroid_Q).T @ (P - centroid_P)   #(3, n) x (n, 3)

        # Use Singular Value Decomposition (SVD) to compute the optimal rotation matrix R
        U, _, Vt = np.linalg.svd(A)
        R = Vt.T @ np.diag([1, 1, np.linalg.det(Vt.T @ U.T)]) @ U.T

        # Compute the optimal translation vector t
        t = centroid_P.T - R @ centroid_Q.T

        # Apply transformation to M2 for the next iteration and store the result
        M2_vertices = (R @ M2_vertices.T + t).T
        registered_model = trimesh.points.PointCloud(M2_vertices)
        registered_models.append(registered_model)

        # Check convergence
        if np.linalg.norm(t) < tolerance:
            print('Converge at iteration', iteration)
            break

    if iteration == max_iterations:
        print(iteration, 'iterations complete.')

    return R, t, registered_models

In [17]:
# Define subsampling rates to test
subsample_rates = [0.1, 0.3, 0.5, 0.7, 0.9]

# Run ICP with subsampling for each subsampling rate
for subsample_rate in subsample_rates:
    print('subsample rate:', subsample_rate)

    _, _, registered_models = icp_with_subsampling(M1, M2, subsample_rate)

    for i, model in enumerate(registered_models):
        directory = f'./results_task2/subsampling_results/subsample_rate_{subsample_rate}/'
        if not os.path.exists(directory):
            os.makedirs(directory)
        filename = f"registered_model_{i+1}.ply"
        filepath = os.path.join(directory, filename)
        model.export(filepath)

subsample rate: 0.1
Converge at iteration 41
subsample rate: 0.3
Converge at iteration 55
subsample rate: 0.5
Converge at iteration 42
subsample rate: 0.7
Converge at iteration 60
subsample rate: 0.9
Converge at iteration 60


### Registration with multiple scans

Now given multiple scans M1,M2,...M5 align all of them to a common global coordinate frame.

In [18]:
# Load point cloud models M1 ~ M5
mesh_fp1 = os.path.join(RES_PATH,'bun000.ply')
assert os.path.exists(mesh_fp1), 'Cannot find:' + mesh_fp1
M1 = trimesh.load(mesh_fp1)
print('M1 shape of Vertices:', M1.vertices.shape)

mesh_fp2 = os.path.join(RES_PATH,'bun045.ply')
assert os.path.exists(mesh_fp2), 'Cannot find:' + mesh_fp2
M2 = trimesh.load(mesh_fp2)
print('M2 shape of Vertices:', M2.vertices.shape)

mesh_fp3 = os.path.join(RES_PATH,'bun090.ply')
assert os.path.exists(mesh_fp3), 'Cannot find:' + mesh_fp3
M3 = trimesh.load(mesh_fp3)
print('M3 shape of Vertices:', M3.vertices.shape)

mesh_fp4 = os.path.join(RES_PATH,'bun180.ply')
assert os.path.exists(mesh_fp4), 'Cannot find:' + mesh_fp4
M4 = trimesh.load(mesh_fp4)
print('M4 shape of Vertices:', M4.vertices.shape)

mesh_fp5 = os.path.join(RES_PATH,'bun270.ply')
assert os.path.exists(mesh_fp5), 'Cannot find:' + mesh_fp5
M5 = trimesh.load(mesh_fp5)
print('M5 shape of Vertices:', M5.vertices.shape)

# Create a list to store models to be registered
models = [M1, M2, M3, M4, M5]
print(models)

M1 shape of Vertices: (40256, 3)
M2 shape of Vertices: (40097, 3)
M3 shape of Vertices: (30379, 3)
M4 shape of Vertices: (40251, 3)
M5 shape of Vertices: (31701, 3)
[<trimesh.PointCloud(vertices.shape=(40256, 3), name=`bun000.ply`)>, <trimesh.PointCloud(vertices.shape=(40097, 3), name=`bun045.ply`)>, <trimesh.PointCloud(vertices.shape=(30379, 3), name=`bun090.ply`)>, <trimesh.PointCloud(vertices.shape=(40251, 3), name=`bun180.ply`)>, <trimesh.PointCloud(vertices.shape=(31701, 3), name=`bun270.ply`)>]


In [19]:
def align_multiple_scans(scans, max_iterations=100, tolerance=1e-6):
    """
    Align multiple scans to a common global coordinate frame.

    Args:
    - scans: a list of numpy arrays representing the vertices of each scan (shape: [N_scan, Nx3])
    - max_iterations: maximum number of iterations for ICP algorithm
    - tolerance: tolerance for convergence

    Returns:
    - aligned_scans: a list of aligned scans (as trimesh.pointcloud.PointCloud objects)
    """

    # Initialize list to store aligned scans
    aligned_scans = []
    aligned_scans.append(scans[0])

    for i, scan in enumerate(scans[1:], start=1):
        print(f'Processing model M{i+1}...')
        # Copy the vertices to avoid modifying the original data
        scan_vertices = scan.vertices.copy()

        # Set reference scan as the previous one
        reference_scan = aligned_scans[i-1].vertices.copy()

        # Initialize rotation matrix R and translation vector t
        R = np.eye(3)
        t = np.zeros((3, 1))

        # Perform ICP to align the current scan to the reference scan
        for iteration in range(1, max_iterations+1):
            # Find the nearest neighbors between the current scan and the reference scan
            tree = KDTree(reference_scan)
            _, indices = tree.query(scan_vertices)

            # Extract corresponding points from the reference scan and the current scan
            P = reference_scan[indices].squeeze()
            Q = scan_vertices

            # Compute the centroid of the matched points
            centroid_P = np.mean(P, axis=0, keepdims=True)  #(1, 3)
            centroid_Q = np.mean(Q, axis=0, keepdims=True)  #(1, 3)

            # Compute the cross-covariance matrix
            A = (Q - centroid_Q).T @ (P - centroid_P)   #(3, n) x (n, 3)

            # Use Singular Value Decomposition (SVD) to compute the optimal rotation matrix R
            U, _, Vt = np.linalg.svd(A)
            R = Vt.T @ np.diag([1, 1, np.linalg.det(Vt.T @ U.T)]) @U.T

            # Compute the optimal translation vector t
            t = centroid_P.T - R @ centroid_Q.T

            # Apply transformation to the current scan for the next iteration
            scan_vertices = (R @ scan_vertices.T + t).T

            # Check convergence
            if np.linalg.norm(t) < tolerance:
                print('Converge at iteration', iteration)
                break

        if iteration == max_iterations:
            print(iteration, 'iterations complete.')

        # Store the aligned scan
        aligned_scan = trimesh.points.PointCloud(scan_vertices)
        aligned_scans.append(aligned_scan)

    return aligned_scans

In [20]:
aligned_scans = align_multiple_scans(models)

for i, aligned_scan in enumerate(aligned_scans):
    directory = './results_task2/multiple_scans/method1/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = f"registered_M{i+1}.ply"
    filepath = os.path.join(directory, filename)
    aligned_scan.export(filepath)

Processing model M2...


Converge at iteration 60
Processing model M3...
Converge at iteration 90
Processing model M4...
100 iterations complete.
Processing model M5...
100 iterations complete.


In [21]:
def align_multiple_scans_2(scans, max_iterations=100, tolerance=1e-6):
    """
    Align multiple scans to a common global coordinate frame.

    Args:
    - scans: a list of numpy arrays representing the vertices of each scan (shape: [N_scan, Nx3])
    - max_iterations: maximum number of iterations for ICP algorithm
    - tolerance: tolerance for convergence

    Returns:
    - aligned_scans: a list of aligned scans (as trimesh.pointcloud.PointCloud objects)
    """

    # Initialize list to store aligned scans
    aligned_scans = []
    aligned_scans.append(scans[0])

    for i, scan in enumerate(scans[1:], start=1):
        print(f'Processing model M{i+1}...')
        # Copy the vertices to avoid modifying the original data
        scan_vertices = scan.vertices.copy()

        # Set reference scan as all the other scans
        other_scans = scans[:i] + scans[i+1:]
        reference_scan = np.concatenate([s.vertices.copy() for s in other_scans])

        # Initialize rotation matrix R and translation vector t
        R = np.eye(3)
        t = np.zeros((3, 1))

        # Perform ICP to align the current scan to the reference scan
        for iteration in range(1, max_iterations+1):
            # Find the nearest neighbors between the current scan and the reference scan
            tree = KDTree(reference_scan)
            _, indices = tree.query(scan_vertices)

            # Extract corresponding points from the reference scan and the current scan
            P = reference_scan[indices].squeeze()
            Q = scan_vertices

            # Compute the centroid of the matched points
            centroid_P = np.mean(P, axis=0, keepdims=True)  #(1, 3)
            centroid_Q = np.mean(Q, axis=0, keepdims=True)  #(1, 3)

            # Compute the cross-covariance matrix
            A = (Q - centroid_Q).T @ (P - centroid_P)   #(3, n) x (n, 3)

            # Use Singular Value Decomposition (SVD) to compute the optimal rotation matrix R
            U, _, Vt = np.linalg.svd(A)
            R = Vt.T @ np.diag([1, 1, np.linalg.det(Vt.T @ U.T)]) @ U.T

            # Compute the optimal translation vector t
            t = centroid_P.T - R @ centroid_Q.T

            # Apply transformation to the current scan for the next iteration
            scan_vertices = (R @ scan_vertices.T + t).T

            # Check convergence
            if np.linalg.norm(t) < tolerance:
                print('Converge at iteration', iteration)
                break

        if iteration == max_iterations:
            print(iteration, 'iterations complete.')

        # Store the aligned scan
        aligned_scan = trimesh.points.PointCloud(scan_vertices)
        aligned_scans.append(aligned_scan)

    return aligned_scans

In [22]:
aligned_scans = align_multiple_scans_2(models)

for i, aligned_scan in enumerate(aligned_scans):
    directory = './results_task2/multiple_scans/method2/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = f"registered_M{i+1}.ply"
    filepath = os.path.join(directory, filename)
    aligned_scan.export(filepath)

Processing model M2...
100 iterations complete.
Processing model M3...
100 iterations complete.
Processing model M4...
100 iterations complete.
Processing model M5...
100 iterations complete.


### Point-to-Plane ICP

Now assume that you have access to normals at each vertex of the meshes. Update your viewer to shade the models based on these normals. Now, use the normal information to improve the ICP performance. you are attempting to solve minimize the following objective function E(R,t) =∑i |(Rpi +t − qi) · nqi |2 instead of minimizing E(R,t) =∑i |(Rpi + t − qi)|2 as implemented in Q1. Note that points pi ∈ M1 and qi ∈ M2 with nqi denoting normal at point qi to the mesh M2. This method is commonly referred to point-to-plane ICP.

In [23]:
def lstsq_plane_fitting(surface_points, k):
    ### fill in this function. normals should have same dimension as surface_points (nx3)
    ### k is the number of neighbours

    num_points, _ = surface_points.shape
    # Construct KDTree based on point samples
    tree = KDTree(surface_points)
    # Get k-neighbours' indices of each point (include itself)
    _, indices = tree.query(surface_points, k)
    # Initialise normals
    normals = np.zeros([num_points, 3])
    for point in range(num_points):
        neighbours = surface_points[indices[point], :]
        (a, b, c), residual, rank, s = np.linalg.lstsq(neighbours, np.ones([k]))
        normal = (a, b, c)
        nn = np.linalg.norm(normal)
        normal /= nn
        normals[point, :] = normal
    
    return normals

In [24]:
# Calculate normals of target model M1
normals = lstsq_plane_fitting(M1.vertices.copy(), 3)
# Store in a new .ply file for visualisation
vertices = M1.vertices.copy()
vertex_normals = normals

# Create PLY header
ply_header = (
    "ply\n"
    "format ascii 1.0\n"
    "element vertex {}\n"
    "property float x\n"
    "property float y\n"
    "property float z\n"
    "property float nx\n"
    "property float ny\n"
    "property float nz\n"
    "end_header\n"
).format(len(vertices))

# Write the vertex coordinates and normal vector to the PLY file
with open("bun000_with_normals.ply", "w") as f:
    f.write(ply_header)
    for i in range(len(vertices)):
        f.write("{} {} {} {} {} {}\n".format(
            vertices[i][0], vertices[i][1], vertices[i][2],
            vertex_normals[i][0], vertex_normals[i][1], vertex_normals[i][2]
        ))

/var/folders/3t/lp7_63n15cxf9p82nfqb28m40000gn/T/ipykernel_63951/3455697768.py:14: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  (a, b, c), residual, rank, s = np.linalg.lstsq(neighbours, np.ones([k]))


In [25]:
def construct_matrix_A_and_b(P, Q, N):
    num_points = P.shape[0]
    A = np.zeros((num_points, 6))
    b = np.zeros((num_points, 1))
    
    for i in range(num_points):
        p_x, p_y, p_z = P[i]
        q_x, q_y, q_z = Q[i]
        n_x, n_y, n_z = N[i]
        A[i, :3] = [n_z * p_y - n_y * p_z, n_x * p_z - n_z * p_x, n_y * p_x - n_x * p_y]
        A[i, 3:] = N[i]
        b[i] = n_x * (q_x - p_x) + n_y * (q_y - p_y) + n_z * (q_z - p_z)
    
    return A, b

def solve_for_transformation(A, b):

    # Initialize rotation matrix R and translation vector t
    R = np.eye(3)
    t = np.zeros((3, 1))

    # Solve least square estimation
    # x = A†b (Moore-Penrose Inverse & SVD)
    # x = [[alpha], [beta], [gamma], [tx], [ty], [tz]], size(6, 1)
    x, _, _, _ = np.linalg.lstsq(A, b, rcond=None)  
    
    # Construct rotation matrix
    [[alpha], [beta], [gamma]] = x[:3]
    r1_1 = np.cos(gamma) * np.cos(beta)
    r1_2 = -np.sin(gamma) * np.cos(alpha) + np.cos(gamma) * np.sin(beta) * np.sin(alpha)
    r1_3 = np.sin(gamma) * np.sin(alpha) + np.cos(gamma) * np.sin(beta) * np.cos(alpha)
    r2_1 = np.sin(gamma) * np.cos(beta)
    r2_2 = np.cos(gamma) * np.cos(alpha) + np.sin(gamma) * np.sin(beta) * np.sin(alpha)
    r2_3 = -np.cos(gamma) * np.sin(alpha) + np.sin(gamma) * np.sin(beta) * np.cos(alpha)
    r3_1 = -np.sin(beta)
    r3_2 = np.cos(beta) * np.sin(alpha)
    r3_3 = np.cos(beta) * np.cos(alpha)
    R = np.array([[r1_1, r1_2, r1_3],
                  [r2_1, r2_2, r2_3],
                  [r3_1, r3_2, r3_3]])

    # Construct translation matrix
    t = x[3:]

    return R, t

def point2plane_icp(M1, M2, max_iterations=100, tolerance=1e-6):

    M1_vertices = M1.vertices.copy()
    M2_vertices = M2.vertices.copy()

    M1_normals = lstsq_plane_fitting(M1_vertices, 3)  # estimate normals of M1's vertices

    # Initialize rotation matrix R and translation vector t
    R = np.eye(3)
    t = np.zeros((3, 1))
    
    # Initialize list to store registered models
    registered_models = []

    for iteration in range(1, max_iterations+1):
        tree = KDTree(M1_vertices)
        distances, indices = tree.query(M2_vertices)

        # if np.mean(distances) < tolerance:
        #     print(f'Convergence achieved after {iteration} iterations.')
        #     break

        P = M2_vertices
        Q = M1_vertices[indices].squeeze()
        N = M1_normals[indices].squeeze()

        A, b = construct_matrix_A_and_b(P, Q, N)
        R, t = solve_for_transformation(A, b)

        # Check convergence
        if np.linalg.norm(t) < tolerance:
            print('Converge at iteration', iteration)
            break
        
        M2_vertices = (R @ M2_vertices.T + t).T
        registered_model = trimesh.points.PointCloud(M2_vertices)
        registered_models.append(registered_model)

    if iteration == max_iterations:
        print(iteration, 'iterations complete.')

    return R, t, registered_models

In [26]:
# Conduct point to plane algorithm...
_, _, registered_models = point2plane_icp(M1, M2)

# Save registered models
for i, model in enumerate(registered_models):
    directory = './results_task2/point2plane_ICP_results/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = f"registered_model_{i+1}.ply"
    filepath = os.path.join(directory, filename)
    model.export(filepath)

/var/folders/3t/lp7_63n15cxf9p82nfqb28m40000gn/T/ipykernel_63951/3455697768.py:14: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  (a, b, c), residual, rank, s = np.linalg.lstsq(neighbours, np.ones([k]))


Converge at iteration 37
